# Dask-traj distributed

This combines dask-traj with dask.distributed in order to show the full capabilities. In this example we use a LocalCluster, but this can be changed out to a distributed Cluster as well (for instructions on how to run distributed on you hpc, have a look at these videos: https://training.e-cam2020.eu/spaces/5cd1521fe4b0fed49058bda6). 

In [1]:
import mdtraj as md
import dask_traj as dmd
import dask
import itertools as itt
import numpy as np
from dask.distributed import Client
from dask.array import allclose

In [2]:
# These files can be provided upon request, 
# but are to big to include in the repository (100 MB)
xtc = '5550217/new.xtc'
pdb = '5550217/new.pdb'
gro = '5550217/new.gro'

# Setup a dask.distributed cluster

Here we will be using a local cluster so we can shortcut the setup by just calling `Client()`

In [3]:
%%time
client = Client()

CPU times: user 345 ms, sys: 32.8 ms, total: 378 ms
Wall time: 655 ms


In [4]:
# Print the client to see the details of the started cluster
client

Client Scheduler: tcp://127.0.0.1:39697 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.42 GB


# Loading trajectories
The (lazy) loading we want to do locally, so we don't do anything different than the other example notebook

There are 2 differences between dmd.load() and md.load().

1.    `dmd.load()` has a keyword chunks that is the number of frames per chunk (naming was copied from dask)
2.    `dmd.load()` is a delayed function, so it does not actually load the files (it just checks the lenght of the file, so a filetype where this is known, like .xtc, would be adviced)


In [5]:
%time dask_traj = dmd.load(xtc, top=gro, chunks=100)

CPU times: user 273 ms, sys: 29.2 ms, total: 302 ms
Wall time: 671 ms


In [6]:
%time mdtraj = md.load(xtc, top=gro)

CPU times: user 1.82 s, sys: 37.9 ms, total: 1.86 s
Wall time: 1.83 s


In [7]:
dask_traj

<mdtraj.Trajectory with 1251 frames, 22561 atoms, 6832 residues, and unitcells at 0x7f057525b5f8>

In [8]:
mdtraj

<mdtraj.Trajectory with 1251 frames, 22561 atoms, 6832 residues, and unitcells at 0x7f0575c84048>

# Doing a computation

First we make some atom pairs

In [9]:
c = list(itt.combinations(range(500),2))
# This does not work anymore for mdtraj (Memory error, depending on the RAM size)
# but should still work for dask_traj
#c = list(itt.combinations(range(3000),2))
print(len(c))

124750


Then do a dask-traj computation with the LocalCluster.

In [10]:
%%time
# First scatter the trajectory to the workers
data  = client.scatter(dask_traj)
# Submit the compute task to the workers 
fut = client.submit(dmd.compute_distances, data, atom_pairs=c, pure=True)
# Get the result from the future
dmd_result = fut.result()
# Actually calculate the result
dmd_out = dmd_result.compute()

CPU times: user 1.45 s, sys: 618 ms, total: 2.07 s
Wall time: 6.32 s


In [11]:
# Get some nice data on the size of our result Array and chunks 
dmd_result

dask.array<concatenate, shape=(1251, 124750), dtype=float32, chunksize=(100, 124750), chunktype=numpy.ndarray>

In [12]:
# Visualize the whole compute graph
dmd_result.visualize(optimize_graph=True)

In [13]:
%%time
# Do the same with mdtraj
md_result = md.compute_distances(mdtraj, atom_pairs=c)

CPU times: user 11.5 s, sys: 75.3 ms, total: 11.6 s
Wall time: 11.5 s


In [14]:
dmd_out

array([[0.1014791 , 0.10119769, 0.10115819, ..., 0.20895208, 0.3013205 ,
        0.10936645],
       [0.10096513, 0.10082649, 0.10140543, ..., 0.2117476 , 0.3043042 ,
        0.10861864],
       [0.10097025, 0.10124216, 0.10097533, ..., 0.21021414, 0.30004996,
        0.10856345],
       ...,
       [0.10058825, 0.1010496 , 0.10097037, ..., 0.21064195, 0.3037204 ,
        0.10966318],
       [0.10127181, 0.10077702, 0.10146917, ..., 0.21887217, 0.30625457,
        0.10951238],
       [0.1007573 , 0.10143467, 0.10112376, ..., 0.21415856, 0.30125386,
        0.1086186 ]], dtype=float32)

In [15]:
md_result

array([[0.1014791 , 0.10119769, 0.10115819, ..., 0.20895208, 0.3013205 ,
        0.10936645],
       [0.10096513, 0.10082649, 0.10140543, ..., 0.2117476 , 0.3043042 ,
        0.10861864],
       [0.10097025, 0.10124216, 0.10097533, ..., 0.21021414, 0.30004996,
        0.10856345],
       ...,
       [0.10058825, 0.1010496 , 0.10097037, ..., 0.21064195, 0.3037204 ,
        0.10966318],
       [0.10127181, 0.10077702, 0.10146917, ..., 0.21887217, 0.30625457,
        0.10951238],
       [0.1007573 , 0.10143467, 0.10112376, ..., 0.21415856, 0.30125386,
        0.1086186 ]], dtype=float32)

In [16]:
# The tollerance is needed due to the round-tripping of unitcell_vectors by mdtraj, but not by dask-traj 
# (see https://github.com/mdtraj/mdtraj/issues/1463)
np.testing.assert_allclose(md_result, dmd_out, atol=2e-6)

## A Fairer timings comparison

As you can see we already are faster with `dask-traj` than `mdtraj`, even if there are an unfavorable amount of chunks versus cores (13 chunks vs. 12 cores) and `mdtraj` is OpenMP enabled as well. 

But just to be fair, we swap the order (to prevent thermal throttling from being an issue), include the loading of the trajectories, and the starting of the clusters with the timings as well. 

In [17]:
client.close() # Close the previous cluster before doing new timings

In [18]:
%%time
mdtraj = md.load(xtc, top=gro)
md_result = md.compute_distances(mdtraj, atom_pairs=c)

CPU times: user 13.2 s, sys: 141 ms, total: 13.3 s
Wall time: 13.3 s


In [19]:
%%time
# Restart client, this is not needed normally!
client = Client()
# Locally (lazy) load the trajectory
dask_traj = dmd.load(xtc, top=gro, chunks=100)
# First scatter the trajectory to the workers
data  = client.scatter(dask_traj)
# Submit the compute task to the workers 
fut = client.submit(dmd.compute_distances, data, atom_pairs=c, pure=True)
# Get the result from the future
dmd_result = fut.result()
# Actually calculate the result
dmd_out = dmd_result.compute()

CPU times: user 1.83 s, sys: 653 ms, total: 2.49 s
Wall time: 7.29 s


In [20]:
np.testing.assert_allclose(md_result, dmd_out, atol=2e-6)

In [21]:
# Proof that a new client was started
client

Client Scheduler: tcp://127.0.0.1:36641 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.42 GB
